## Extract Seasonality variables from boxoffice website

In [2]:
!pip install Selenium

     |████████████████████████████████| 911kB 3.6MB/s 


In [0]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
import os
import time
import pandas as pd
import re
from datetime import datetime, timedelta

In [0]:
import requests
import urllib
import urllib.request
import time
import re
from bs4 import BeautifulSoup
from IPython.core.display import HTML

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import copy
import datetime 
from collections import Counter

import selenium
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
def cutGrossNum(string_check, start_ = "$", end_=["+", "-"]):
  gross = ""
  loc = string_check.find(start_) + 1
  for j in range(loc, len(string_check)):
    if string_check[j] in end_:
      gross = gross.replace(",", "")
      return int(gross)
    else:
      gross = gross + string_check[j]
  # return none value find
  return np.nan

In [35]:
filled_set = pd.read_csv("Film-masterfile (from 'film-merge-8th.dta')-filled.csv")
filled_set['releasedate'] =  pd.to_datetime(filled_set['releasedate'])
filled_set['releasedate'].head(5)

0   2013-09-20
1   2012-07-12
2   2012-09-07
3   2014-08-29
4   2015-10-21
Name: releasedate, dtype: datetime64[ns]

In [38]:
# simple test to exact date info extraction
print(filled_set['releasedate'].min())
print(filled_set['releasedate'].min().year)
print(filled_set['releasedate'].min().month)
print(filled_set['releasedate'].max())

2004-01-09 00:00:00
2004
1
2015-12-30 00:00:00


In [190]:
# create a dataframe to record all info from boxoffice 
# initialization a single year dataset
WEEKS = 52
YEAR_START = 2002
YEAR_END = 2015
columns_name = [ 'year' , 'week', 'boxOfficeUrl', 'dateString',
                'startDate', 'endDate', 'gross', 'totalMovies']
dataframe_boxoffice_year = pd.DataFrame(index=np.arange(WEEKS), columns=columns_name)
# initialize the dateframe for all years collected
dataframe_boxoffice = pd.DataFrame()
for index_ in range(YEAR_START, YEAR_END+1):
  dataframe_boxoffice_year['year'] = index_
  dataframe_boxoffice_year['week'] = range(1, WEEKS+1)
  dataframe_boxoffice = dataframe_boxoffice.append(dataframe_boxoffice_year)
# reset_index
dataframe_boxoffice = dataframe_boxoffice.reset_index(drop=True)

# check shape 
print(dataframe_boxoffice.shape)
dataframe_boxoffice.head(5)

(728, 8)


,year,week,boxOfficeUrl,dateString,startDate,endDate,gross,totalMovies
0,2002,1,NaN,NaN,NaN,NaN,NaN,NaN
1,2002,2,NaN,NaN,NaN,NaN,NaN,NaN
2,2002,3,NaN,NaN,NaN,NaN,NaN,NaN
3,2002,4,NaN,NaN,NaN,NaN,NaN,NaN
4,2002,5,NaN,NaN,NaN,NaN,NaN,NaN


In [191]:
# fill in the url column by year and week column
dataframe_boxoffice['year'] = dataframe_boxoffice['year'].apply(str)
dataframe_boxoffice['week'] = dataframe_boxoffice['week'].apply(str)
dataframe_boxoffice['boxOfficeUrl'] = "https://www.boxofficemojo.com/weekly/chart/?yr=" + \
                                       dataframe_boxoffice['year'] + "&wk=" + \
                                       dataframe_boxoffice['week'] + "&p=.htm"
print(dataframe_boxoffice['boxOfficeUrl'][100])
dataframe_boxoffice.head(5)

https://www.boxofficemojo.com/weekly/chart/?yr=2003&wk=49&p=.htm


,year,week,boxOfficeUrl,dateString,startDate,endDate,gross,totalMovies
0,2002,1,https://www.boxofficemojo.com/weekly/chart/?yr...,NaN,NaN,NaN,NaN,NaN
1,2002,2,https://www.boxofficemojo.com/weekly/chart/?yr...,NaN,NaN,NaN,NaN,NaN
2,2002,3,https://www.boxofficemojo.com/weekly/chart/?yr...,NaN,NaN,NaN,NaN,NaN
3,2002,4,https://www.boxofficemojo.com/weekly/chart/?yr...,NaN,NaN,NaN,NaN,NaN
4,2002,5,https://www.boxofficemojo.com/weekly/chart/?yr...,NaN,NaN,NaN,NaN,NaN


In [192]:
# Now fill in the dateString and gross and totalMovies column
keywords = "TOTAL"
for ii in range(dataframe_boxoffice.shape[0]):
  # extract url address
  boxOffice_ = dataframe_boxoffice['boxOfficeUrl'][ii]
  # extract page source for each url address
  response = requests.get(boxOffice_)
  soup = BeautifulSoup(response.text, "html.parser")

  # fill in dateString
  dataframe_boxoffice['dateString'][ii] = soup.find("title").text
  # fill in gross and total number of movies 
  filter_string = soup.find_all("tr", attrs={"bgcolor":"#dcdcdc"})
  for jj in range(len(filter_string)):
    if keywords in filter_string[jj].text:
      # cut movie number here
      dataframe_boxoffice['totalMovies'][ii] = int(filter_string[jj].text.split(" ")[1][1:])
      dataframe_boxoffice['gross'][ii] = cutGrossNum(filter_string[jj].text)
      break
  # process
  if ii%100 == 0:
    print(ii)

0
100
200
300
400
500
600
700


In [193]:
# fill in start date and end date
dataframe_boxoffice.head(5)

,year,week,boxOfficeUrl,dateString,startDate,endDate,gross,totalMovies
0,2002,1,https://www.boxofficemojo.com/weekly/chart/?yr...,"Weekly Box Office Results for January 4-10, 20...",NaN,NaN,160260760,54
1,2002,2,https://www.boxofficemojo.com/weekly/chart/?yr...,"Weekly Box Office Results for January 11-17, 2...",NaN,NaN,138189002,54
2,2002,3,https://www.boxofficemojo.com/weekly/chart/?yr...,"Weekly Box Office Results for January 18-24, 2...",NaN,NaN,176703388,48
3,2002,4,https://www.boxofficemojo.com/weekly/chart/?yr...,"Weekly Box Office Results for January 25-31, 2...",NaN,NaN,164197937,55
4,2002,5,https://www.boxofficemojo.com/weekly/chart/?yr...,"Weekly Box Office Results for February 1-7, 20...",NaN,NaN,125959322,56


In [194]:
# check if all dateString contains date info
dataframe_boxoffice[~dataframe_boxoffice['dateString'].str.contains('\d', regex=True)]

,year,week,boxOfficeUrl,dateString,startDate,endDate,gross,totalMovies


In [0]:
# deal with dateString column function
def timeBoundExtract(dateString_, length_ = 4, keywords_ = "December"):
  string = dateString_.replace("Weekly Box Office Results for ", "").replace(" - Box Office Mojo", "")
  # separate year and month-date info
  year_string = string[-4:]
  month_date_string = string[:-6]
  temp_list = month_date_string.split("-")

  if len(temp_list[1]) >= length_:
    # this type contains both month name in two elements
    if keywords_ in temp_list[0]:
      # here means across the year
      temp_list[1] = temp_list[1] + " " + year_string
      temp_list[0] = temp_list[0] + " " + str(int(year_string)-1)
    else:
      # here means simply across the month
      temp_list[0] = temp_list[0] + " " + year_string
      temp_list[1] = temp_list[1] + " " + year_string
  else:
    # week simply within same month-year
    month_string = temp_list[0].split(" ")[0]
    temp_list[0] = temp_list[0] + " " + year_string
    temp_list[1] = month_string + " " + temp_list[1] + " " + year_string
  # turn temp_list into timestamp finally
  temp_list[0] = pd.to_datetime(temp_list[0], infer_datetime_format=True)
  temp_list[1] = pd.to_datetime(temp_list[1], infer_datetime_format=True)
  return temp_list

In [196]:
for ii in range(dataframe_boxoffice.shape[0]):
  content = dataframe_boxoffice['dateString'][ii]
  time_list = timeBoundExtract(content)
  dataframe_boxoffice['startDate'][ii] = time_list[0]
  dataframe_boxoffice['endDate'][ii] = time_list[1]
  # process
  if ii%100 == 0:
    print(ii)

0
100
200
300
400
500
600
700


In [197]:
dataframe_boxoffice.head(20)

,year,week,boxOfficeUrl,dateString,startDate,endDate,gross,totalMovies
0,2002,1,https://www.boxofficemojo.com/weekly/chart/?yr...,"Weekly Box Office Results for January 4-10, 20...",2002-01-04 00:00:00,2002-01-10 00:00:00,160260760,54
1,2002,2,https://www.boxofficemojo.com/weekly/chart/?yr...,"Weekly Box Office Results for January 11-17, 2...",2002-01-11 00:00:00,2002-01-17 00:00:00,138189002,54
2,2002,3,https://www.boxofficemojo.com/weekly/chart/?yr...,"Weekly Box Office Results for January 18-24, 2...",2002-01-18 00:00:00,2002-01-24 00:00:00,176703388,48
3,2002,4,https://www.boxofficemojo.com/weekly/chart/?yr...,"Weekly Box Office Results for January 25-31, 2...",2002-01-25 00:00:00,2002-01-31 00:00:00,164197937,55
4,2002,5,https://www.boxofficemojo.com/weekly/chart/?yr...,"Weekly Box Office Results for February 1-7, 20...",2002-02-01 00:00:00,2002-02-07 00:00:00,125959322,56
5,2002,6,https://www.boxofficemojo.com/weekly/chart/?yr...,"Weekly Box Office Results for February 8-14, 2...",2002-02-08 00:00:00,2002-02-14 00:00:00,135636421,57
6,2002,7,https://www.boxofficemojo.com/weekly/chart/?yr...,"Weekly Box Office Results for February 15-21, ...",2002-02-15 00:00:00,2002-02-21 00:00:00,192176483,56
7,2002,8,https://www.boxofficemojo.com/weekly/chart/?yr...,"Weekly Box Office Results for February 22-28, ...",2002-02-22 00:00:00,2002-02-28 00:00:00,135869548,58
8,2002,9,https://www.boxofficemojo.com/weekly/chart/?yr...,"Weekly Box Office Results for March 1-7, 2002 ...",2002-03-01 00:00:00,2002-03-07 00:00:00,133835495,60
9,2002,10,https://www.boxofficemojo.com/weekly/chart/?yr...,"Weekly Box Office Results for March 8-14, 2002...",2002-03-08 00:00:00,2002-03-14 00:00:00,138475523,61


In [0]:
dataframe_boxoffice.to_csv("boxoffice_weekly_gross_movie.csv")